In [1]:
import warnings
from fastai.vision import *
from fastai.callbacks import *
from fastai.vision import models 
from fastai.vision.models import cadene_models 
warnings.filterwarnings('ignore')
path = Path('../data/histopathologic/')
sys.path.append("dev/"); from metric_utils import AUC

In [2]:
# create ensemble dir
os.makedirs(path/'ensemble', exist_ok=True)

In [3]:
preds_path = path/'preds'
subs_path = path/'submissions'
ens_path = path/'ensemble'

In [4]:
preds_best_of_dirs = (preds_path).ls(); preds_best_of_dirs

[PosixPath('../data/histopathologic/preds/best_of_resnet152_non_overlap'),
 PosixPath('../data/histopathologic/preds/best_of_densenet201_non_overlap'),
 PosixPath('../data/histopathologic/preds/best_of_densenet169_non_overlap'),
 PosixPath('../data/histopathologic/preds/best_of_dpn92_non_overlap'),
 PosixPath('../data/histopathologic/preds/best_of_resnet101_non_overlap')]

In [5]:
(preds_path/'best_of_resnet152_non_overlap').ls()

[PosixPath('../data/histopathologic/preds/best_of_resnet152_non_overlap/fold4_preds.pkl'),
 PosixPath('../data/histopathologic/preds/best_of_resnet152_non_overlap/fold1_TTA_preds.pkl'),
 PosixPath('../data/histopathologic/preds/best_of_resnet152_non_overlap/fold2_TTA_preds.pkl'),
 PosixPath('../data/histopathologic/preds/best_of_resnet152_non_overlap/fold2_preds.pkl'),
 PosixPath('../data/histopathologic/preds/best_of_resnet152_non_overlap/fold0_TTA_preds.pkl'),
 PosixPath('../data/histopathologic/preds/best_of_resnet152_non_overlap/fold3_TTA_preds.pkl'),
 PosixPath('../data/histopathologic/preds/best_of_resnet152_non_overlap/fold3_preds.pkl'),
 PosixPath('../data/histopathologic/preds/best_of_resnet152_non_overlap/fold1_preds.pkl'),
 PosixPath('../data/histopathologic/preds/best_of_resnet152_non_overlap/fold0_preds.pkl'),
 PosixPath('../data/histopathologic/preds/best_of_resnet152_non_overlap/fold4_TTA_preds.pkl')]

In [8]:
[o.name.strip("best_of_") for o in preds_best_of_dirs]

['resnet152_non_overlap',
 'densenet201_non_overlap',
 'densenet169_non_overlap',
 'dpn92_non_overlap',
 'resnet101_non_overlap']

### simple average

In [23]:
models = [o.name.strip("best_of_") for o in preds_best_of_dirs]
models  = ['resnet152_non_overlap',
 'densenet201_non_overlap',
 'dpn92_non_overlap']

In [24]:
mean_label = np.mean([pd.read_csv(subs_path/f'best_of_{m}/{m}_avg_TTA.csv')['label'] for m
           in models], axis=0)

In [25]:
new_sub = pd.read_csv(subs_path/f'best_of_{models[0]}/{models[0]}_avg_TTA.csv')
new_sub['label'] = mean_label

In [26]:
fname = "_".join(models)
new_sub.to_csv(ens_path/f"avg_TTA_mean_{fname}", index=False)

In [27]:
sub_name = ens_path/f"avg_TTA_mean_{fname}"

In [28]:
sub_name, new_sub

(PosixPath('../data/histopathologic/ensemble/avg_TTA_mean_resnet152_non_overlap_densenet201_non_overlap_dpn92_non_overlap_resnet101_non_overlap'),
                                              id     label
 0      0b2ea2a822ad23fdb1b5dd26653da899fbd2c0d5  0.005553
 1      95596b92e5066c5c52466c90b69ff089b39f2737  0.090460
 2      248e6738860e2ebcf6258cdc1f32f299e0c76914  0.000990
 3      2c35657e312966e9294eac6841726ff3a748febf  0.017133
 4      145782eb7caa1c516acbe2eda34d9a3f31c41fd6  0.026611
 5      725dabe6ecccc68b958a2c7dd75bcbf362c7cb03  0.943567
 6      aa0307865281d4484ddf8c637c348292968b93a7  0.416751
 7      f4e5dc9c949920f1b3362982e15e99bf6f3ef83b  0.304668
 8      95e08c9cedc28a9b4a86f4fc1e06c1972134be08  0.008162
 9      ee1f5345a8d4e28403b7e61b97b5f76c201ce9cb  0.025626
 10     14b1e9a72e27a35f13056db611feb3c8825916f4  0.866853
 11     4f7710b24b9b776f6bfc89fad5469592d18e5431  0.006693
 12     2b0668109c7c446c94d29e1b554016b486e84bde  0.001228
 13     8dbd712b5b5e86c18c9

In [29]:
cmd = "kaggle competitions submit -c histopathologic-cancer-detection -f {} -m {}_avg_TTA".format(sub_name, fname)
os.system(cmd), print(cmd)

kaggle competitions submit -c histopathologic-cancer-detection -f ../data/histopathologic/ensemble/avg_TTA_mean_resnet152_non_overlap_densenet201_non_overlap_dpn92_non_overlap_resnet101_non_overlap -m resnet152_non_overlap_densenet201_non_overlap_dpn92_non_overlap_resnet101_non_overlap_avg_TTA


(0, None)